In [1]:
# librerias

import pandas as pd
import numpy as np
import collections as lc
import itertools as li


In [5]:
# definir un grafo

# Informacion: Recordar que en un grafo se conectan nodos:  nodo_A -> nodo_B
# en algunos casos no es lo mismo nodo_A -> nodo_B  != nodo_B -> nodo_A ... son nodos direccionados
# también estan los grafos que no posee direccion de conexion


### Grafos sin direccion ####
# crear un nuevo grafo ... se indica "nodo" : "nodos conectados"
# NOTA: en caso se tenga: "nodo": {} ; significa que es un nodo aislado, no posee ninguna conexion a otro nodo

graph = { "a" : {"c"},
          "b" : {"c", "e"},
          "c" : {"a", "b", "d", "e"},
          "d" : {"c"},
          "e" : {"c", "b"},
          "f" : {}
}

# Ver grafo
graph

{'a': {'c'},
 'b': {'c', 'e'},
 'c': {'a', 'b', 'd', 'e'},
 'd': {'c'},
 'e': {'b', 'c'},
 'f': {}}

In [7]:
# generar conexion entre nodos (sin direccion)

# funcion para generar conexiones entre los nodos segun grafo (diccionario) indicado
def generate_edges(g):
    edges = []
    for node in g:
        for vecino in g[node]:
            # agrega un conjunto (set()) .. no es diccionario es set
            edges.append({node,vecino})
    return edges

# calcular las conexiones
generate_edges(graph)

[{'a', 'c'},
 {'b', 'c'},
 {'b', 'e'},
 {'c', 'd'},
 {'a', 'c'},
 {'c', 'e'},
 {'b', 'c'},
 {'c', 'd'},
 {'c', 'e'},
 {'b', 'e'}]

In [8]:
# encontrar los nodos aislados

# buscar nodos aislados , aquellos que no poseen conexion a ningun nodo
def find_isolated_nodes(g):
    isolated = set()
    for node in g:
        if not g[node]:
            isolated.add(node)
    return isolated

find_isolated_nodes(graph)

{'f'}

In [13]:
# Clase para el maenjo de grafos

# definir clase
class Graph(object):
    
    # inicializar el grafo
    def __init__(self, graph_dict=None):
        # en caso no exista grafo aun
        if(graph_dict is None):
            graph_dict = {}
        # en caso ya exista el grafo
        self._graph_dict = graph_dict
    
    
    # conexiones 
    def edges(self,vertice):
        return self._graph_dict[vertice]
    
    
    # extraer todos los nodos existentes en el grafo
    def all_vertices(self):
        return set(self._graph_dict.keys())
    
    
    # extraer todas las conexiones del grafo
    def all_edges(self):
        return self.__generate_edges()
    
    
    # añadir un nodo
    def add_vertex(self,vertex):
        if vertex not in self._graph_dict:
            self._graph_dict[vertex]=[]
    
    
    # añadir una conexion
    def add_edge(self,edge):
        edge = set(edge)
        v1,v2 = tuple(edge)
        for x,y in [(v1,v2),(v2,v1)]:
            if x in self._graph_dict:
                self._graph_dict[x].add(y)
            else:
                self._graph_dict[x] = {y}
    
    
    # generar todas las conexiones
    def __generate_edges(self):
        edges = []
        for vertex in self._graph_dict:
            for vecino in self._graph_dict[vertex]:
                if {vecino,vertex} not in edges:
                    edges.append({vertex,vecino})
        return edges
    
    
    # crear elemento "iterador" para ir recorriendo elemento a elemento
    def __iter__(self):
        self.iter_obj = iter(self._graph_dict)
        return self.iter_obj
    
    
    # funcion que permite recorrer los elementos del "iterador" (previo)
    def __next__(self):
        return next(self.iter_obj)
    
    
    # imprimir el grafo ... recordar esta es la funcion magica "str"
    def __str__(self):
        res = "nodos: "
        for k in self._graph_dict:
            res += str(k) + " "
        res += "\nconexion: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res
    
    
    # Encontrar el camino que va de "nodo_inicio" a "nodo_fin", encuentra los nodos del camino 
    def find_path(self,start_vertex, end_vertex, path=None):
        
        # si no hay camino --- condicion de inicio
        if path == None:
            # asignar lista vacia
            path = []
            
        # variable para guardar el diccionario
        graph = self._graph_dict
        
        # Se añade el punto de partida
        path = path + [start_vertex]
        
        # en caso coincida el punto inicio y fin se retorna el camino encontrado
        ### Condicion de final ###
        if start_vertex == end_vertex:
            return path
        
        # en caso el vertice de inicio no exista en el diccionario
        ### Condicion de final ###
        if start_vertex not in graph:
            # regresa nada, porque no existe camino
            return None
        
        # en otro caso, que si exista este vertice en el diccionario, recorre elementos
        for vertex in graph[start_vertex]:
            # en caso este vertice no exista en "path"
            if vertex not in path:
                
                # encontrar el camino de este vertice hasta el final
                ### Condicion de recurrencia ###
                extended_path = self.find_path(vertex,end_vertex,path)
                
                if extended_path:
                    # en caso de no ser vacio ... devolver dicho camino
                    return extended_path
                
        # en otro caso devuelve vacio o nulo
        return None
    
    
    # Buscar todos los caminos o trayectorias que llevan de "nodo_inicio" a "nodo_fin"
    def find_all_paths(self,start_vertex,end_vertex,path=[]):
        
        # guardar el diccionario con el grafo
        graph = self._graph_dict
        
        # agregar el primer vertice al camino
        path = path + [start_vertex]
        
        # en caso de ser igual el inicio al fin
        ### Condicion de final ###
        if start_vertex == end_vertex:
            # retornar el camino hasta el momento
            return [path]
        
        # en caso el vertice no este en el diccionario
        ### Condicion de final ###
        if start_vertex not in graph:
            # no hay camino
            return []
        paths = []
        
        # recorrer los vertices segun diccionario
        for vertex in graph[start_vertex]:
            
            # en caso el vertice no este en el camino actual
            if vertex not in path:
                
                # calcular los caminos de este vertice hacia el final
                ### Condicion de recurrencia ###
                extended_paths = self.find_all_paths(vertex, end_vertex,path)
                
                # agregar los caminos encontrados 
                for p in extended_paths:
                    paths.append(p)
                    
        # retornar los caminos encontrados
        return paths

# fin de definicion de clase

In [14]:
# usar la clase

g = { "a" : {"d"},
      "b" : {"c"},
      "c" : {"b", "c", "d", "e"},
      "d" : {"a", "c"},
      "e" : {"c"},
      "f" : {}
    }

graph = Graph(g)

for vertice in graph:
    print(f"Conexiones de nodos {vertice}: ", graph.edges(vertice))
print('-----------------------------')
print(graph)

v1 = 'a'   
v2 = 'e'
print('Camino desde ' + v1 + ' a ' + v2 + ' : ',graph.find_path(v1,v2))
print('Todos los caminos desde ' + v1 + ' a ' + v2 + ' : ',graph.find_path(v1,v2))
print('-----------------------------')

v1 = 'd'   
v2 = 'c'
print('Camino desde ' + v1 + ' a ' + v2 + ' : ',graph.find_path(v1,v2))
print('All Paths from ' + v1 + ' a ' + v2 + ' : ',graph.find_path(v1,v2))
print('-----------------------------')

v1 = 'c'   
v2 = 'c'
print('Camino desde ' + v1 + ' a ' + v2 + ' : ',graph.find_path(v1,v2))
print('Todos los caminos desde ' + v1 + ' a ' + v2 + ' : ',graph.find_path(v1,v2))

# fin

Conexiones de nodos a:  {'d'}
Conexiones de nodos b:  {'c'}
Conexiones de nodos c:  {'e', 'd', 'c', 'b'}
Conexiones de nodos d:  {'a', 'c'}
Conexiones de nodos e:  {'c'}
Conexiones de nodos f:  {}
-----------------------------
nodos: a b c d e f 
conexion: {'d', 'a'} {'c', 'b'} {'c', 'e'} {'d', 'c'} {'c'} 
Camino desde a a e :  ['a', 'd', 'c', 'e']
Todos los caminos desde a a e :  ['a', 'd', 'c', 'e']
-----------------------------
Camino desde d a c :  ['d', 'c']
All Paths from d a c :  ['d', 'c']
-----------------------------
Camino desde c a c :  ['c']
Todos los caminos desde c a c :  ['c']
